In [1]:
from pyspark import SparkContext, SparkConf
import numpy as np

#Configure pyspark
conf = SparkConf().setMaster("local[*]")
sc = SparkContext(conf=conf)

In [4]:
#Read in the text file as an RDD
rdd = sc.textFile('T10I4D100K.dat')

baskets = rdd.map(lambda line: line.split(" ")[0:len(line.split(" ")) - 1]).cache()
count = baskets.count()
flattened_baskets = baskets.flatMap(lambda line: line)
occurences = flattened_baskets.map(lambda x: (int(x),1)).reduceByKey(lambda a,b: a+b)

L1 = occurences.filter(lambda x: x[1] > count*0.01).sortByKey(True).collect()
L1[0:5]

[(1, 1535), (4, 1394), (5, 1094), (6, 2149), (8, 3090)]

In [5]:
#Go through L1 and find frequent items
frequent_items = np.zeros(count + 1)
for elem in L1:
    frequent_items[elem[0]] = 1
frequent_items[1:5]

array([1., 0., 0., 1.])

In [15]:
def doubletons(doc):
    pairs = []
    for i in range(0,len(doc)-1):
        for k in range(i+1,len(doc)-1):
            if frequent_items[int(doc[i])] == 1 and frequent_items[int(doc[i+1])] == 1:
                pairs.append(doc[i] + " " + doc[k])
    return pairs
L2 = baskets.map(doubletons).flatMap(lambda x: x).map(lambda x: (x,1)
                                                             ).reduceByKey(lambda a,b: a+b)
L2.take(5)

[('25 368', 97), ('25 448', 18), ('25 630', 15), ('25 687', 12)]